<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GEE-Authentication" data-toc-modified-id="GEE-Authentication-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>GEE Authentication</a></span></li><li><span><a href="#Map-Loading" data-toc-modified-id="Map-Loading-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Map Loading</a></span><ul class="toc-item"><li><span><a href="#Hardcoded-Inputs-(Change-the-values-in-this)" data-toc-modified-id="Hardcoded-Inputs-(Change-the-values-in-this)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Hardcoded Inputs (Change the values in this)</a></span></li><li><span><a href="#Data-Extraction-and-Visualization" data-toc-modified-id="Data-Extraction-and-Visualization-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data Extraction and Visualization</a></span></li></ul></li><li><span><a href="#Labelling-and-Subsequent-Exportation" data-toc-modified-id="Labelling-and-Subsequent-Exportation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Labelling and Subsequent Exportation</a></span></li></ul></div>

# GEE Authentication

Users will first need to authenticate and initialize GEE's Python API based on the cell below using their gmail. The steps required to do so will be listed when the cell below is run.

In [ ]:
# Importing all modules needed for this notebook
%reset -f
import os
import ee
import geemap
import pandas as pd

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Map Loading

This section then reads data based off *EventsList.xlsx* in the Data folder for either fire or lava images across both satellite types. Loaded images can then be annotated on (labelling), where both the labelled data and the image can be saved thereafter in the following section. All images are based off the surface reflectance products of [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) and [Landsat 8](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2).

Lava images are defined based on centering the loaded image on the Volcano's coordinates, while fire images are defined based on centering the loaded image of a given burnt area. The difference however, is that fire images are further specified based on how the satellite splits Earth into 100 x 100$km^{2}$ grids (or tiles), where the grids can be accessed from this .kml file [here](https://sentinel.esa.int/documents/247904/1955685/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml).

In other words, the grid naming convention in Sentinel-2 **does not** apply to Landsat 8, whereby if one were to introduce Landsat 8 data for fire images in the future, it would have to use an entirely different grid system (see ***Figure 3 of [this paper](https://www.researchgate.net/publication/317095580_Combined_Use_of_Landsat-8_and_Sentinel-2A_Images_for_Winter_Crop_Mapping_and_Winter_Wheat_Yield_Assessment_at_Regional_Scale)*** as an example). 

## Hardcoded Inputs (Change the values in this)

In [ ]:
# The main directory in which data is kept
homeDir = r'C:\Users\ansel\Desktop\MSS Internship Stuff\Data'

# The satellite and image type (lava or fire) for which we want to extract data for.
    # Currently, the dataset (from EventsList.xlsx) only contains S2 Fire, S2 Lava, and LS8 Lava images 
satellite = "S2"
hotspot = 'fire'

# [FIXED] Ensuring we only look at 5km x 5km images at all times
bufferRange = 5000

## Data Extraction and Visualization

In [ ]:
# Finding the path where the excel file is kept
    # i.e., ...\Data\EventsList.xlsx
excelPath = os.path.join(homeDir,'EventsList.xlsx')

# Defining the destination directory in which files will eventually be saved
    # i.e., ...\Data\Lava Images\S2\...
destDir = os.path.join(homeDir,hotspot.capitalize() + 'Images',satellite)

# Reading excel database as a dataframe
# Creating a new column that represents potential subfolder (child directory) names
# Finding the smallest index where the child directory value of ['childDir'] does not exist as a true directory
    # So that the code knows the event corresponding to this child directory has not been read yet
    # Template for Lava images follow (Volcano followed by date):
        # i.e., ...\Data\Lava Images\LS8\Agung-20180728
    # Tempalte for fire images (only S2) follow (S2 Grid followed by date and number of images for that same grid and date):
        # i.e., ...\Data\Fire Images\S2\46QEJ-20190320[0]
if hotspot == 'lava':
    df = pd.read_excel(excelPath,sheet_name=satellite,header=0)
    df['Date'] = df['Date'].astype(str) # Converting timestamp object to string
    df['childDir'] = homeDir + '\\' + hotspot.capitalize() + '\\' + satellite + '\\' + df['Volcano'].apply(lambda v: v.capitalize()) + '-' + df['Date'].replace('-','',regex=True)
elif hotspot == 'fire':
    df = pd.read_excel(excelPath,sheet_name=hotspot.capitalize(),header=0)
    df['Date'] = df['Date'].astype(str) # Converting timestamp object to string
    df['childDir'] = homeDir + '\\' + hotspot.capitalize() + '\\' + satellite + '\\' + df['Tile'] + '-' + df['Date'].replace('-','',regex=True) + '[' + df['Count'].astype(str) + ']'

    
# Finding the index of the file to be read (a file whose childDir does not exist yet)
    # idx is therefore the most important variable here as all other data required for loading the map rests on this variable.
pathBool = df['childDir'].apply(os.path.isdir)
if pathBool.sum() == len(df.index):
    idx = 0
    # raise KeyboardInterrupt('All files have already been read and downloaded before.')
else:
    idx = pathBool.idxmin()
childDir = df.loc[idx,'childDir']


# Defining key details necessary for data extraction.
    # Tile name and corresponding burnt area lat/lon for fire images
    # Volcano and corresponding lat/lon for lava images
if hotspot == 'fire':
    Lon = df.loc[idx,'Lon']
    Lat = df.loc[idx,'Lat']
    tile = df.loc[idx,'Tile']
elif hotspot == 'lava':
    coords = {'agung':(-8.343,115.508),
             'bromo':(-7.942,112.95),
             'dukono':(1.7,127.8667),
             'gamalama':(0.8,127.33),
             'ibu':(1.488,127.63),
             'karangetang':(2.781,125.407),
             'kerinci':(-1.697,101.233),
             'krakatau':(-6.1021,105.4230),
             'lewotolo':(-8.3,123.5050),
             'marapi':(-0.38,100.4742),
             'merapi':(-7.5407,110.4457),
             'mayon':(13.257,123.685),
             'raung':(-8.119,114.056),
             'sangeangapi':(-8.2,119.07),
             'sinabung':(3.1696,98.3930),
             'sirung':(-8.508,124.13),
             'semeru':(-8.1077,112.9224),
             'soputan':(1.11298,124.72916),
             'taal':(14.0113,120.9977),
             'hthh':(-20.536,-175.382),
             'pinatubo':(15.1429,120.3496)}
    volc = df.loc[idx,'Volcano']
    Lon = coords[volc][1]
    Lat = coords[volc][0]


# Loading the date of the event.
date = df.loc[idx,'Date']    
startDate = '%sT00:00:00' % date
endDate = '%sT23:59:00' % date
    
# First identifying the area of interest (aoi), and generating a base map for the image to be displayed on later using geemap.
aoi = ee.Geometry.Point(Lon, Lat)
Map = geemap.Map(center=[Lat,Lon], zoom=13)


# Bands in common between LS8 & S2
    # S2 - B1, B2, B3, B4, B8A, B11, B12, NDVI, NBR, IWCD
    # LS8 - SR_B1, SR_B2, SR_B3, SR_B4, SR_B5, SR_B6, SR_B7, NDVI, NBR, IWCD
if satellite == 'S2':
    S2_features = ['B1', 'B2', 'B3', 'B4', 'B8A', 'B11', 'B12', 'NDVI', 'NBR', 'IWCD']
    
    # Different extraction method for lava and fire images (latter also requires specifying the MGRS_TILE)
        # Regardless of hotspot type, we scale all the bands first before we define the 3 feature engineered bands (NDVI, NBR & IWCD)
    if hotspot == 'lava':
        image = ee.ImageCollection('COPERNICUS/S2_SR')\
                  .filterBounds(aoi)\
                  .filterDate(startDate,endDate)\
                  .map(lambda img: img.divide(10000).copyProperties(img,img.propertyNames()))\
                  .map(lambda img: img.addBands(img.normalizedDifference(['B8','B4']).rename('NDVI')))\
                  .map(lambda img: img.addBands(img.normalizedDifference(['B8','B12']).rename('NBR')))\
                  .map(lambda img: img.addBands(img.select('B4').subtract(img.select('B11')).rename('IWCD')))\
                  .first().select(S2_features)
    elif hotspot == 'fire':
        image = ee.ImageCollection('COPERNICUS/S2_SR')\
                  .filterMetadata('MGRS_TILE','equals',tile)\
                  .filterBounds(aoi)\
                  .filterDate(startDate,endDate)\
                  .map(lambda img: img.divide(10000).copyProperties(img,img.propertyNames()))\
                  .map(lambda img: img.addBands(img.normalizedDifference(['B8','B4']).rename('NDVI')))\
                  .map(lambda img: img.addBands(img.normalizedDifference(['B8','B12']).rename('NBR')))\
                  .map(lambda img: img.addBands(img.select('B4').subtract(img.select('B11')).rename('IWCD')))\
                  .first().select(S2_features)
    
    # Color stretching is fixed to enable consistent display of images for all S2 images.
    Map.addLayer(image.clip(aoi.buffer(bufferRange)),
                 {'bands':['B12','B8A','B4'],'min':0,'max':1.6},
                 '0 to 1.6 range')
    

elif satellite == 'LS8':
    LS8_features = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'NDVI', 'NBR', 'IWCD']
    
    # Only for lava images
        # Regardless of hotspot type, we scale all the bands first before we define the 3 feature engineered bands (NDVI, NBR & IWCD)
    image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
              .filterBounds(aoi)\
              .filterDate(startDate,endDate)\
              .map(lambda img: img.addBands(img.select('SR_B.').multiply(0.0000275).add(-0.2), None, True).copyProperties(img,img.propertyNames()))\
              .map(lambda img: img.addBands(img.normalizedDifference(['SR_B5','SR_B4']).rename('NDVI')))\
              .map(lambda img: img.addBands(img.normalizedDifference(['SR_B5','SR_B7']).rename('NBR')))\
              .map(lambda img: img.addBands(img.select('SR_B4').subtract(img.select('SR_B6')).rename('IWCD')))\
              .first().select(LS8_features)
    
    # Color stretching is fixed to enable consistent display of images for all LS8 images.
    Map.addLayer(image.clip(aoi.buffer(bufferRange)),
                 {'bands':['SR_B7','SR_B5','SR_B4'],'min':-0.003,'max':0.75},
                 '-0.003 to 0.75 range')
    
else:
    raise ValueError('Select the right satellite - "S2" or "LS8".')

# Before the map is displayed, the event of interest is printed for better readability.
if hotspot == 'lava':
    print('Lava Image: %s, %s' % (volc.capitalize(),date))
elif hotspot == 'fire':
    print('Fire Image: Tile %s, %s, Image #%d' % (tile,date,df.loc[idx,'Count']))
    
# Displaying the selected image.
display(Map)

# Labelling and Subsequent Exportation

After you have loaded the map above, you can then proceed to label the clear pixels as individual polygons (see **[this tutorial](https://www.youtube.com/watch?v=N7rK2aV1R4c&list=PLAxJ4-o7ZoPccOFv1dCwvGI6TYnirRTg3&index=7&ab_channel=QiushengWu)** for drawing tools on geemap).

The cell below will then label the polygons in the order that they were drawn, where then these polygons will be exported as a vector (.shp) file and the image as a raster (.tif) file thereafter (see **[this tutorial](https://www.youtube.com/watch?v=_6JOA-iiEGU&list=PLAxJ4-o7ZoPccOFv1dCwvGI6TYnirRTg3&index=13&ab_channel=QiushengWu)** for exporting data using GEE and geemap).

In [ ]:
# MUST CHANGE THIS FOR EVERY LABELLED IMAGE
    # label the polygons in the order that they were required, i.e., ['fire', 'fire', 'fire', 'vegetation', 'cloud', 'shadow']
classes = []

# Convert drawn polygons above to a feature collection
shapesMap = ee.FeatureCollection(Map.draw_features)

# A sanity check to ensure the number of labelled classes equals the number of polygons created
if len(classes) != shapesMap.size().getInfo():
    raise ValueError('Number of classes defined (%d) not equals to total polygons drawn (%d).' % 
                    (len(classes), shapesMap.size().getInfo()))
    
# Converting featurecollection to list of properties, and zipping values of classes to each feature (list of 2-element lists)
shapesZipped = shapesMap.toList(shapesMap.size()).zip(ee.List(classes))

# Recreating the polygons by combining each 2-element list (feature + new class value) into a new feature collection
shapes = ee.FeatureCollection(shapesZipped.map(lambda l: ee.Feature(ee.List(l).get(0)).set('class',ee.List(l).get(1))))

# Defining paths for the vector and raster files, and also creating the child directories if they don't exist
rasterfileOut = os.path.join(childDir,"raster.tif")
shapefileOut = os.path.join(childDir,"vector.shp")
if not os.path.isdir(childDir):
    os.mkdir(childDir)
    
# Saving the resultant feature collection
geemap.ee_to_shp(shapes, shapefileOut)

# Saving the raster file too
    # Fixed scale of 30m for both Sentinel-2 and Landsat 8 data for standardization purposes.
    # All 10 bands are saved as a single .tif file
    # Only saving a 5km x 5km area centered on either the burnt area (for fire images) or the volcano (for lava images)
geemap.ee_export_image(image, filename = rasterfileOut, scale = 30, 
                       region = aoi.buffer(bufferRange), file_per_band = False)